# Get vaccine coverage by ZIP Codes data from CDPH

In [2]:
import pandas as pd
import datetime as dt
import json
import os
import glob
import urllib.request

In [3]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
pd.set_option("display.max_colwidth", None)

In [4]:
today = dt.datetime.today().strftime("%Y-%m-%d")

### Get the metadata from the API

In [5]:
endpoint = "https://data.chhs.ca.gov/api/3/action/package_show?id=covid-19-vaccine-progress-dashboard-data-by-zip-code"

In [6]:
jsonurl = urllib.request.urlopen(endpoint)
text = json.loads(jsonurl.read())

### Get the object with the max date

In [22]:
data = text["result"]["resources"]

### Read that latest file into a dataframe to limit redacted rows ~~and save it in raw~~

In [23]:
url = "https://data.chhs.ca.gov/dataset/ead44d40-fd63-4f9f-950a-3b0111074de8/resource/ec32eece-7474-4488-87f0-6e91cb577458/download/covid19vaccinesbyzipcode_test.csv"

In [24]:
df = pd.read_csv(url)

In [25]:
df = df[df["vem_source"] != "No VEM Assigned"]

In [26]:
df = df.rename(columns={"as_of_date": "date"})

## Export

In [27]:
df[df.zip_code_tabulation_area == 90012].tail()

,date,zip_code_tabulation_area,local_health_jurisdiction,county,vaccine_equity_metric_quartile,vem_source,age12_plus_population,age5_plus_population,persons_fully_vaccinated,persons_partially_vaccinated,percent_of_population_fully_vaccinated,percent_of_population_partially_vaccinated,percent_of_population_with_1_plus_dose,redacted
84587,2021-11-30,90012,Los Angeles,Los Angeles,1.0,Healthy Places Index Score,34607.1,35495,52.0,29.0,0.001465,0.000817,0.002282,No
85523,2021-12-07,90012,Los Angeles,Los Angeles,1.0,Healthy Places Index Score,34607.1,35495,52.0,30.0,0.001465,0.000845,0.002310,No
87513,2021-12-14,90012,Los Angeles,Los Angeles,1.0,Healthy Places Index Score,34607.1,35495,52.0,31.0,0.001465,0.000873,0.002338,No
89163,2021-12-21,90012,Los Angeles,Los Angeles,1.0,Healthy Places Index Score,34607.1,35495,52.0,31.0,0.001465,0.000873,0.002338,No
90770,2021-12-28,90012,Los Angeles,Los Angeles,1.0,Healthy Places Index Score,34607.1,35495,52.0,43.0,0.001465,0.001211,0.002676,No


### All updates

In [13]:
df.to_csv("data/timeseries.csv", index=False)

### Latest update

In [14]:
df[df["date"] == df["date"].max()].to_csv("data/latest.csv", index=False)